In [6]:
# Importing the Keras libraries and packages
import keras
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
import numpy as np

In [2]:
# Initialising the CNN
classifier = Sequential()

# Step 1 - Convolution
classifier.add(Convolution2D(32, 3, 3, input_shape = (64, 64, 3), activation = 'relu'))

# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Adding a second convolutional layer
classifier.add(Convolution2D(32, 3, 3, activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Step 3 - Flattening
classifier.add(Flatten())

# Step 4 - Full connection
classifier.add(Dense(output_dim = 128, activation = 'relu'))
classifier.add(Dense(output_dim = 5, activation = 'softmax'))

# Compiling the CNN
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:5: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu", input_shape=(64, 64, 3...)`
  """
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:11: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu")`
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:18: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=128, activation="relu")`
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:19: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=5, activation="softmax")`


In [3]:
# Part 2 - Fitting the CNN to the images

from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('train',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')
label_map = (training_set.class_indices)

test_set = test_datagen.flow_from_directory('test',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'categorical')

classifier.fit_generator(training_set,
                         samples_per_epoch = 8000,
                         nb_epoch = 30,
                         validation_data = test_set,
                         nb_val_samples = 800)

Found 2939 images belonging to 5 classes.
Found 731 images belonging to 5 classes.


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:27: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:27: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., validation_data=<keras_pre..., steps_per_epoch=250, epochs=30, validation_steps=800)`


Epoch 1/30
250/250 [==============================] - 154s 614ms/step - loss: 1.1326 - acc: 0.5306 - val_loss: 1.0006 - val_acc: 0.6131
Epoch 2/30
250/250 [==============================] - 113s 452ms/step - loss: 0.8738 - acc: 0.6574 - val_loss: 0.8395 - val_acc: 0.6742
Epoch 3/30
250/250 [==============================] - 115s 461ms/step - loss: 0.7674 - acc: 0.6970 - val_loss: 0.7924 - val_acc: 0.7082
Epoch 4/30
250/250 [==============================] - 118s 471ms/step - loss: 0.6521 - acc: 0.7489 - val_loss: 0.8561 - val_acc: 0.6858
Epoch 5/30
250/250 [==============================] - 116s 463ms/step - loss: 0.5691 - acc: 0.7741 - val_loss: 0.8017 - val_acc: 0.7099
Epoch 6/30
250/250 [==============================] - 117s 470ms/step - loss: 0.4953 - acc: 0.8107 - val_loss: 0.8970 - val_acc: 0.7075
Epoch 7/30
250/250 [==============================] - 119s 476ms/step - loss: 0.4114 - acc: 0.8460 - val_loss: 0.8224 - val_acc: 0.7176
Epoch 8/30
250/250 [============================

In [4]:
# Predicions on test set
test_set.reset()
pred = classifier.predict_generator(test_set, verbose=1)

23/23 [==============================] - 2s 85ms/step


In [7]:
predicted_class_indices=np.argmax(pred,axis=1)

In [8]:
labels= training_set.class_indices
labels = dict((k,v) for v,k in labels.items())
print(labels)

{0: 'daisy', 1: 'dandelion', 2: 'roses', 3: 'sunflowers', 4: 'tulips'}


In [9]:
predictions = [labels[k] for k in predicted_class_indices]

In [10]:
import pandas as pd
name = test_set.filenames
to_save = pd.DataFrame({'filenames': name, 'pred':predictions})
to_save.to_csv('pred.csv', index = False)